## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [30]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from math import ceil

# Завантаження даних
df = pd.read_csv("cookie_cats.csv")
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [10]:
effect_size = sms.proportion_effectsize(0.2, 0.19)    # Розрахунок розміру ефекту на основі очікуваних показників
effect_size

np.float64(0.025241594409087353)

In [27]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа
required_n

24638

Щоб мати можливість виявити зростання утримання на 1 % (з 19% → 20%) з імовірністю 80% (потужність), необхідно зібрати дані приблизно по 24638 користувачів в кожній з груп.

2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

###Гіпотези:

**H₀ (нульова гіпотеза):** Середнє утримання на 7-й день однакове для обох версій гри (gate_30 = gate_40).

**H₁ (альтернативна гіпотеза):** Середнє утримання на 7-й день відрізняється між версіями.


In [45]:
ret7_stats = df.groupby('version')['retention_7'].mean()
ret7_stats

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [42]:
# Вибираємо дані по групах
gate_30 = df[df.version == 'gate_30']['retention_7']
gate_40 = df[df.version == 'gate_40']['retention_7']

# Кількість користувачів у кожній групі
n_gate_30 = gate_30.count()
n_gate_40 = gate_40.count()

# Кількість "успіхів" (користувач повернувся на 7-й день)
successes = [gate_30.sum(), gate_40.sum()]

# Кількість спостережень у кожній групі
nobs = [n_gate_30, n_gate_40]

# Чіткий друк результату
print(f"Група gate_30: успіхів = {successes[0]}, розмір вибірки = {nobs[0]}")
print(f"Група gate_40: успіхів = {successes[1]}, розмір вибірки = {nobs[1]}")

Група gate_30: успіхів = 8502, розмір вибірки = 44700
Група gate_40: успіхів = 8279, розмір вибірки = 45489


In [44]:
# Z-тест
z_stat, pval = proportions_ztest(count, nobs=nobs)

# Довірчі інтервали 95% для кожної групи
(ci_lower_30, ci_lower_40), (ci_upper_30, ci_upper_40) = proportion_confint(count, nobs=nobs, alpha=0.05)

# Вивід результатів
print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи gate_30: [{ci_lower_30:.3f}, {ci_upper_30:.3f}]')
print(f'Довірчий інтервал 95% для групи gate_40: [{ci_lower_40:.3f}, {ci_upper_40:.3f}]')

z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи gate_30: [0.187, 0.194]
Довірчий інтервал 95% для групи gate_40: [0.178, 0.186]


###Висновок A/B тесту

- **p-значення = 0.002 < 0.05** → відхиляємо нульову гіпотезу H0.  
- Є статистично значуща різниця між групами.  
- **Утримання на 7-й день:**
  - Control (gate_30): **18.7% – 19.4%**  
  - Treatment (gate_40): **17.8% – 18.6%**  

✅ Переміщення воріт на 40-й рівень **суттєво вплинуло на утримання**, проте у цьому випадку контрольна група показала **вищий retention**, ніж тестова. Тобто зміна розташування воріт **знизила утримання на 7-й день**.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


###Гіпотези:
**H₀ (нульова гіпотеза):** Версія гри не впливає на утримання гравців на 7-й день (немає залежності між групою і retention).

**H₁ (альтернативна гіпотеза):** Версія гри впливає на утримання гравців на 7-й день (існує залежність між групою і retention).

In [25]:
crosstab = pd.crosstab(df['version'], df['retention_7'])

In [26]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


Оскільки p-value < 0.05, існує статистично значущий зв’язок між групою користувача (gate_30 / gate_40) і показником retention на 7-й день.